2.1-Análisis de rango de valores por columna del dataset MOVIES
-

In [ ]:
#Se crea un dataset auxiliar que se usará en algunos analisis
movies_analisys = movies.copy()

In [ ]:
# Columna "belongs_to_collection"
print("Describe de la columna belongs_to_collection: ")
print(movies.belongs_to_collection.describe())


Describe de la columna belongs_to_collection: 
count                                                  4488
unique                                                 1695
top       {'id': 415931, 'name': 'The Bowery Boys', 'pos...
freq                                                     29
Name: belongs_to_collection, dtype: object


In [ ]:
# Generamos un listado pelicula - coleccion
movies[['title','belongs_to_collection']][movies['belongs_to_collection'].notna()].head()

,title,belongs_to_collection
0,Toy Story,Toy Story Collection
2,Grumpier Old Men,Grumpy Old Men Collection
4,Father of the Bride Part II,Father of the Bride Collection
9,GoldenEye,James Bond Collection
12,Balto,Balto Collection


In [ ]:
# Identificamos el numero de colecciones y las colecciones que mas peliculas tienen
movies_by_collection = movies[['belongs_to_collection','id']].groupby(['belongs_to_collection']).count()
movies_by_collection
print('Hay',movies_by_collection.id.count(),'colecciones diferentes')
movies_by_collection.sort_values(by='id', ascending = False).head(10)

Hay 1695 colecciones diferentes


,id
belongs_to_collection,
The Bowery Boys,29
Totò Collection,27
James Bond Collection,26
Zatôichi: The Blind Swordsman,26
The Carry On Collection,25
Pokémon Collection,22
Charlie Chan (Sidney Toler) Collection,21
Godzilla (Showa) Collection,16
Uuno Turhapuro,15


In [ ]:
# Columna "budget"
print("Valor minimo y maximo de la columna budget. Min: ",movies.budget.min(), "Max", movies.budget.max(), movies.budget.dtypes)


Valor minimo y maximo de la columna budget. Min:  0 Max 998000 object


In [ ]:
# Columna "id"
print("Valor minimo y maximo de la columna id. Min: ",movies.id.min(), "Max", movies.id.max(), movies.id.dtypes)

Valor minimo y maximo de la columna id. Min:  100 Max 9999 object


In [ ]:
# Los minimos y maximos se estan calculando en sentido alfabetico. Se convierte la variable en numerica
movies['id'] = movies.id.astype(int)
print("Valor minimo y maximo de la columna id. Min: ",movies.id.min(), "Max", movies.id.max(), movies.id.dtypes)

Valor minimo y maximo de la columna id. Min:  2 Max 469172 int32


In [ ]:
# Columna "genres"
print("Describe de la columna genres: ")
print(movies.genres.describe())

Describe de la columna genres: 
count                             45376
unique                             4065
top       [{'id': 18, 'name': 'Drama'}]
freq                               4998
Name: genres, dtype: object


In [ ]:
# Columna "original_language"
print("Describe de la columna original_language: ")
print(movies.original_language.describe())
print(movies.original_language.unique())


Describe de la columna original_language: 
count     45365
unique       89
top          en
freq      32202
Name: original_language, dtype: object
['en' 'fr' 'zh' 'it' 'fa' 'nl' 'de' 'cn' 'ar' 'es' 'ru' 'sv' 'ja' 'ko'
 'sr' 'bn' 'he' 'pt' 'wo' 'ro' 'hu' 'cy' 'vi' 'cs' 'da' 'no' 'nb' 'pl'
 'el' 'sh' 'xx' 'mk' 'bo' 'ca' 'fi' 'th' 'sk' 'bs' 'hi' 'tr' 'is' 'ps'
 'ab' 'eo' 'ka' 'mn' 'bm' 'zu' 'uk' 'af' 'la' 'et' 'ku' 'fy' 'lv' 'ta'
 'sl' 'tl' 'ur' 'rw' 'id' 'bg' 'mr' 'lt' 'kk' 'ms' 'sq' nan 'qu' 'te' 'am'
 'jv' 'tg' 'ml' 'hr' 'lo' 'ay' 'kn' 'eu' 'ne' 'pa' 'ky' 'gl' 'uz' 'sm'
 'mt' 'hy' 'iu' 'lb' 'si']


In [ ]:
# Columna "popularity"
print("Describe de la columna popularity: ")
print(movies.popularity.describe())

Describe de la columna popularity: 
count     45376
unique    44142
top       1e-06
freq         30
Name: popularity, dtype: object


In [ ]:
#Se verifica si la columna popularity contiene outliers: valores muy alejados de la media:
#Para no usar Numpy definimos una funcion que pueda encontra el percentil y asi encontrar los cuartiles 1 y 3
def percentil(per, datos):
    # Copiar la lista para no alterarla
    x = datos.copy()
    x.sort()

    n = len(x)
    rank = (per / 100.0) * (n - 1)
    rank_int = int(rank)
    rank_dec = rank % 1

    if rank_dec:
        p = x[rank_int] + rank_dec * (x[rank_int + 1] - x[rank_int])
    else:
        p = x[rank_int]

    return p

# Usamos la funcion definida para encontrar el Q1 y el Q3
Q1 = percentil(25, movies.popularity.to_list())
Q3 = percentil(75, movies.popularity.to_list())
# Calculamos el rango intercuartil
IQR = Q3-Q1

print ('Popularity: Primer cuartil',Q1, 'Tercer cuartil',Q3, 'Rango intercuartil', IQR)

movies_analisys['popularity_outlayer'] = movies_analisys['popularity'].apply(lambda x: (x < Q1-(1.5*IQR) or x > Q3+(1.5*IQR)))

movies_analisys[['popularity_outlayer','popularity']].groupby(['popularity_outlayer']).count()


Popularity: Primer cuartil 0.3888395 Tercer cuartil 3.6916945 Rango intercuartil 3.302855


,popularity
popularity_outlayer,
False,41168
True,4208


In [ ]:
# Revisando las 4208 peliculas cuya popularidad son outlayers
popularity_analisys = movies_analisys[['title','popularity']][movies_analisys['popularity_outlayer']==True]
popularity_analisys = popularity_analisys.sort_values(by=['popularity'], ascending=False)
print(popularity_analisys)

                      title  popularity
30700               Minions  547.488298
33356          Wonder Woman  294.337037
42222  Beauty and the Beast  287.253654
43644           Baby Driver  228.032744
24455            Big Hero 6  213.849907
...                     ...         ...
15087       Valhalla Rising    8.651468
25981                Tarzan    8.651247
22046                   Ida    8.651053
5677         Half Past Dead    8.650691
34068             Safelight    8.649580

[4208 rows x 2 columns]


In [ ]:
 #Despues de ver los outliers en popularidad, se considera que se deben conservar tal como estan

# Columna "release_date"
print("Describe de la columna release_date: ")
print(movies.release_date.describe())

Describe de la columna release_date: 
count          45376
unique         17333
top       2008-01-01
freq             136
Name: release_date, dtype: object


In [ ]:
# Columna "revenue"
print("Describe de la columna revenue: ")
print(movies.revenue.describe())


Describe de la columna revenue: 
count    4.537600e+04
mean     1.123010e+07
std      6.438996e+07
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      0.000000e+00
max      2.787965e+09
Name: revenue, dtype: float64


In [ ]:
# Generamos una sumatoria del revenue por año
temp_movies=movies[['revenue','release_year']].groupby(['release_year']).sum()
temp_movies = temp_movies.sort_values(by=['release_year'], ascending=False)

print(temp_movies)

                   revenue
release_year              
2020          0.000000e+00
2018          0.000000e+00
2017          1.507589e+10
2016          3.049788e+10
2015          2.881936e+10
...                    ...
1888          0.000000e+00
1887          0.000000e+00
1883          0.000000e+00
1878          0.000000e+00
1874          0.000000e+00

[135 rows x 1 columns]


In [ ]:
# El revenue por año parece consistente con el numero de peliculas por año

In [ ]:
# Columna "runtime"
print("Describe de la columna runtime: ")
print(movies.runtime.describe())

Describe de la columna runtime: 
count    45130.000000
mean        94.181675
std         38.341059
min          0.000000
25%         85.000000
50%         95.000000
75%        107.000000
max       1256.000000
Name: runtime, dtype: float64


In [ ]:
# Columna "status"
print("Describe de la columna status: ")
print(movies.status.describe())

Describe de la columna status: 
count        45296
unique           6
top       Released
freq         44936
Name: status, dtype: object


In [ ]:
# Generamos la lista unica de estados y su conteo
temp_movies=movies[['id','status']].groupby(['status']).count()
temp_movies = temp_movies.sort_values(by=['status'], ascending=False)

print(temp_movies)

                    id
status                
Rumored            230
Released         44936
Post Production     97
Planned             13
In Production       19
Canceled             1


In [ ]:
# Columna "title"
print("Describe de la columna title: ")
print(movies.title.describe())

Describe de la columna title: 
count          44936
unique         41813
top       Cinderella
freq              11
Name: title, dtype: object


In [ ]:
# El conteo total es mucho mayor que el conteo de únicos. Generamos un  conteo por titulo
temp_movies=movies[['id','title']].groupby(['title']).count()
temp_movies = temp_movies.sort_values(by=['id'], ascending=False)

print(temp_movies.head(20))

                               id
title                            
Cinderella                     11
Alice in Wonderland             9
Hamlet                          9
Beauty and the Beast            8
Les Misérables                  8
The Three Musketeers            7
A Christmas Carol               7
Treasure Island                 7
Blackout                        7
Heidi                           6
Love                            6
The Hound of the Baskervilles   6
Countdown                       6
Macbeth                         6
Home                            6
The Journey                     6
The Stranger                    6
The Hunters                     6
The Promise                     6
The Circle                      6


In [ ]:
# Analizamos las 3 peliculas mas repetidas por título
movies[movies['title'].isin(['Cinderella','Alice in Wonderland', 'Hamlet'])][['id','title','budget','original_language','popularity','release_date','revenue','runtime','status','vote_average','vote_count']].sort_values(['title','release_date'])


,id,title,budget,original_language,popularity,release_date,revenue,runtime,status,vote_average,vote_count
39530,41278,Alice in Wonderland,0,en,0.708323,1903-10-17,0.000000e+00,9.0,Released,6.1,20.0
32901,87300,Alice in Wonderland,0,en,0.140291,1915-01-15,0.000000e+00,52.0,Released,5.5,2.0
15868,25694,Alice in Wonderland,0,en,1.504355,1933-12-22,0.000000e+00,76.0,Released,5.4,8.0
1003,12092,Alice in Wonderland,3000000,en,16.488937,1951-07-03,5.720000e+08,75.0,Released,7.0,1557.0
26894,35109,Alice in Wonderland,0,en,1.716036,1966-12-26,0.000000e+00,72.0,Released,6.9,8.0
34695,84774,Alice in Wonderland,0,ru,0.320034,1981-01-31,0.000000e+00,30.0,Released,8.7,6.0
11887,34573,Alice in Wonderland,14000000,en,3.794133,1985-12-09,0.000000e+00,187.0,Released,6.3,15.0
9665,30923,Alice in Wonderland,21000000,en,8.614303,1999-02-28,0.000000e+00,150.0,Released,6.1,71.0
14892,12155,Alice in Wonderland,200000000,en,17.285093,2010-03-03,1.025491e+09,108.0,Released,6.4,4773.0
41074,114108,Cinderella,0,fr,1.301063,1899-10-01,0.000000e+00,5.0,Released,5.5,19.0


In [ ]:
# De una misma pelicula se hacen multiples versiones. 

#Hay dos peliculas Hamlet en el año 2000. Generamos una lista por pelicula por año , tratando de identificar si hay peliculas duplicadas
temp_movies=movies[movies['title'].isin(['Cinderella','Alice in Wonderland', 'Hamlet'])][['id','title','release_year']].groupby(['title','release_year']).count()
temp_movies.sort_values('id',ascending=False)

id
title               release_year    
Hamlet              2000           2
Alice in Wonderland 1903           1
                    1915           1
Hamlet              1996           1
                    1990           1
                    1980           1
                    1969           1
                    1964           1
                    1948           1
Cinderella          2015           1
                    2012           1
                    2011           1
                    2002           1
                    2000           1
                    1997           1
                    1957           1
                    1950           1
                    1947           1
                    1914           1
                    1899           1
Alice in Wonderland 2010           1
                    1999           1
                    1985           1
                    1981           1
                    1966           1
                    1951           1
                    1933           1
Hamlet              2009           1

In [ ]:
#Solo Hamlet esta repetida para el año 2000. Revisamos en detalle estas dos lineas
movies.query('release_year  == 2000 & title == "Hamlet"')[['id','title','budget','original_language','popularity','release_date','revenue','runtime','status','vote_average','vote_count','overview']]

,id,title,budget,original_language,popularity,release_date,revenue,runtime,status,vote_average,vote_count,overview
3476,10688,Hamlet,2000000,en,12.896203,2000-05-12,1568749.0,112.0,Released,6.0,35.0,Modern day adaptation of Shakespeare's immorta...
13313,125705,Hamlet,0,en,0.584251,2000-12-10,0.0,178.0,Released,0.0,0.0,"To be or not to be, etc."


In [ ]:
# La pelicula Hamlet con id 125705 no tiene presupuesto, revenue, ni votos...

# Buscando otras peliculas que tengas las mismas caracteristicas
movies.query('budget  == 0 & revenue == 0.0 & vote_average == 0.0 & vote_count == 0.0')[['id','title','budget','original_language','popularity','release_date','revenue','runtime','status','vote_average','vote_count','overview']]


,id,title,budget,original_language,popularity,release_date,revenue,runtime,status,vote_average,vote_count,overview
83,188588,Last Summer in the Hamptons,0,en,0.531159,1995-11-22,0.0,108.0,Released,0.0,0.0,"Filmed entirely on location in East Hampton, L..."
107,96357,Headless Body in Topless Bar,0,en,0.001346,1995-05-20,0.0,110.0,Released,0.0,0.0,An ex-con holds a group of people hostage in a...
126,290157,Jupiter's Wife,0,en,0.001178,1995-01-01,0.0,87.0,Released,0.0,0.0,"Michel Negroponte, a documentary filmmaker, me..."
132,124636,Sonic Outlaws,0,en,0.001838,1995-08-01,0.0,87.0,Released,0.0,0.0,Within days after the release of Negativland's...
137,124639,Target,0,en,0.001205,1995-08-01,0.0,122.0,Released,0.0,0.0,A subtle yet violent commentary on feudal lords.
...,...,...,...,...,...,...,...,...,...,...,...,...
45432,323132,Altar of Fire,0,en,0.000000,1976-01-01,0.0,45.0,Released,0.0,0.0,This film records a 12 day ritual performed by...
45434,325439,The Wonders of Aladdin,0,en,0.093498,1961-10-31,0.0,93.0,Released,0.0,0.0,Young Aladdin (Donald O'Connor) has a series o...
45452,276895,Deep Hearts,0,en,0.011025,1981-01-01,0.0,58.0,Released,0.0,0.0,"Deep Hearts is a film about the Bororo Fulani,..."
45464,227506,Satan Triumphant,0,en,0.003503,1917-10-21,0.0,87.0,Released,0.0,0.0,"In a small town live two brothers, one a minis..."


In [ ]:
# Sucede lo mismo en 2684 peliculas. No es buena idea borrarlas

# Columna "vote_average"
print("Describe de la columna vote_average: ")
print(movies.vote_average.describe())

Describe de la columna vote_average: 
count    44936.000000
mean         5.629380
std          1.907455
min          0.000000
25%          5.000000
50%          6.000000
75%          6.800000
max         10.000000
Name: vote_average, dtype: float64


In [ ]:
#Vote average es un valor entre 0 y 10 presente en la mayoria del dataset

# Columna "vote_count"
print("Describe de la columna vote_count: ")
print(movies.vote_count.describe())

Describe de la columna vote_count: 
count    44936.000000
mean       111.079179
std        494.036416
min          0.000000
25%          3.000000
50%         10.000000
75%         35.000000
max      14075.000000
Name: vote_count, dtype: float64
